In [2]:
import pandas as pd
import numpy as np
import random, math

In [482]:
random.seed(786)
np.random.seed(786)

In [483]:
df = pd.read_csv('final_data.csv')
# df = df[['compound','neg','neu','pos','Close']]
df = df[['neg','neu','pos','Close']]

In [484]:
from sklearn import preprocessing
min_max_scalar=preprocessing.MinMaxScaler()
data=min_max_scalar.fit_transform(df)
# X_test=min_max_scalar.fit_transform(X_test)
# Y_train=min_max_scalar.fit_transform(Y_train)
# Y_test=min_max_scalar.fit_transform(Y_test)

In [485]:
# df['Close']=df['Close']/400

In [486]:
# data = df.values.tolist()

In [487]:
data = data.tolist()

In [488]:
inner_threshold = 0.005;
rate_var = 0;

final_ansr = []

In [489]:
sigmoid=lambda x:1/(1+math.e**(-x))

In [490]:
def fit_func(rec, i):
    x1, x2, x3, y = i[0], i[1], i[2], i[3]

    out4 = (x1 * rec['data']['w14']) + (x2 * rec['data']['w24']) + (x3 * rec['data']['w34']) + rec['data']['o4']
    out4 = sigmoid(out4)

    out5 = (x1 * rec['data']['w15']) + (x2 * rec['data']['w25']) + (x3 * rec['data']['w35']) + rec['data']['o5']
    out5 = sigmoid(out5)

    out6 = (out4 * rec['data']['w46']) + (out5 * rec['data']['w56']) + rec['data']['o6']
    out6 = sigmoid(out6)

    err6 = out6 * (1 - out6) * (y - out6)

    if abs(err6) <= inner_threshold:
        # final_ansr.append(out6 * 20000)
        rec['fitness'] += 1


In [491]:
nList = [];
m = 20;
n = 25;
mutation_prob = 75;
mutation = 0.5;

In [492]:
# ranges for weights & biases
wl = -1;
wr = 1;
bl =0;
br = 1

In [493]:
for i in range(n):
    # floating random weights
    w14, w15, w24, w25, w34, w35, w46, w56 = np.random.uniform(wl, wr, 8)
    # floating random biases
    o4, o5, o6 = np.random.uniform(bl, br, 3)

    nList.append({'data': {'w14': w14, 'w15': w15, 'w24': w24, 'w25': w25, 'w34': w34, 'w35': w35, 'w46': w46,
                           'w56': w56, 'o4': o4, 'o5': o5, 'o6': o6}, 'fitness': 0})

In [494]:
# fitness calculation for first time/initial population
for rec in nList:
    for i in data:
        fit_func(rec, i)
        
    print(rec['fitness'])

174
29
172
173
178
25
60
50
112
24
46
62
59
177
29
113
24
19
55
22
174
28
102
51
28


In [495]:
count = 0;
final_rec = 0
flag = False
stop_at=1000

In [496]:
while (1):
    count += 1
    mList = []

    print(f"gen: {count}")

    if count > stop_at:
        final_rec = sorted(nList, key=lambda f: f['fitness'], reverse=True)[0]
        break

    rec_num = 0
    for rec in nList:
        # print("rec_index: {}".format(rec_num))
        rec_num += 1
        if rec['fitness'] == len(data):
            print("rec_index: {}".format(rec_num))
            final_rec = rec
            # print('gen: {}'.format(count))
            print('Answer: {}'.format(rec))
            flag = True
            break

    if flag:
        break

    for j in range(int(m / 2)):
        # randomly parent selection(not same)
        # t=temp

        t = random.sample(range(n), 2)

        p1 = nList[t[0]]
        p2 = nList[t[1]]

        # crossover
        ch1 = list(p1['data'].values())[:5] + list(p2['data'].values())[5:]
        ch2 = list(p2['data'].values())[:5] + list(p1['data'].values())[5:]

        # mutation work
        for k in [ch1, ch2]:
            if random.randint(0, 100) <= mutation_prob:
                # get random for mutation in weights
                t = random.sample(range(8), 3)
                # get random for mutation in bias
                t1 = random.randint(8, 10)
                # odd,do +ve
                if random.randint(1, 2) == 1:
                    # mutation in weights
                    if (k[t[0]] + mutation) >= wl and (k[t[0]] + mutation) <= wr:
                        k[t[0]] += mutation

                    if (k[t[1]] + mutation) >= wl and (k[t[1]] + mutation) <= wr:
                        k[t[1]] += mutation

                    if (k[t[2]] + mutation) >= wl and (k[t[2]] + mutation) <= wr:
                        k[t[2]] += mutation

                    # mutation in bias
                    if (k[t1] + mutation) >= bl and (k[t1] + mutation) <= br:
                        k[t1] += mutation

                # even,do -ve
                else:
                    # mutation in weights
                    if (k[t[0]] - mutation) >= wl and (k[t[0]] - mutation) <= wr:
                        k[t[0]] -= mutation

                    if (k[t[1]] - mutation) >= wl and (k[t[1]] - mutation) <= wr:
                        k[t[1]] -= mutation

                    if (k[t[2]] - mutation) >= wl and (k[t[2]] - mutation) <= wr:
                        k[t[2]] -= mutation

                    # mutation in bias
                    if (k[t1] - mutation) >= bl and (k[t1] - mutation) <= br:
                        k[t1] -= mutation

        ch1 = {'data': {'w14': ch1[0], 'w15': ch1[1], 'w24': ch1[2], 'w25': ch1[3], 'w34': ch1[4], 'w35': ch1[5],
                        'w46': ch1[6],
                        'w56': ch1[7], 'o4': ch1[8], 'o5': ch1[9], 'o6': ch1[10]}, 'fitness': 0}

        ch2 = {'data': {'w14': ch2[0], 'w15': ch2[1], 'w24': ch2[2], 'w25': ch2[3], 'w34': ch2[4], 'w35': ch2[5],
                        'w46': ch2[6],
                        'w56': ch2[7], 'o4': ch2[8], 'o5': ch2[9], 'o6': ch2[10]}, 'fitness': 0}

        # calculate fitness of children
        for rec in [ch1, ch2]:
            for i in data:
                fit_func(rec, i)

        mList.append(ch1)
        mList.append(ch2)

    # combine both mList and nList and sort it with respect fit func value,also ovverides the nList,so we used in next iteration

    nList = sorted(nList + mList, key=lambda item: item['fitness'], reverse=True)[:25]


gen: 1
gen: 2
gen: 3
gen: 4
gen: 5
gen: 6
gen: 7
gen: 8
gen: 9
gen: 10
gen: 11
gen: 12
gen: 13
gen: 14
gen: 15
gen: 16
gen: 17
gen: 18
gen: 19
gen: 20
gen: 21
gen: 22
gen: 23
gen: 24
gen: 25
gen: 26
gen: 27
gen: 28
gen: 29
gen: 30
gen: 31
gen: 32
gen: 33
gen: 34
gen: 35
gen: 36
gen: 37
gen: 38
gen: 39
gen: 40
gen: 41
gen: 42
gen: 43
gen: 44
gen: 45
gen: 46
gen: 47
gen: 48
gen: 49
gen: 50
gen: 51
gen: 52
gen: 53
gen: 54
gen: 55
gen: 56
gen: 57
gen: 58
gen: 59
gen: 60
gen: 61
gen: 62
gen: 63
gen: 64
gen: 65
gen: 66
gen: 67
gen: 68
gen: 69
gen: 70
gen: 71
gen: 72
gen: 73
gen: 74
gen: 75
gen: 76
gen: 77
gen: 78
gen: 79
gen: 80
gen: 81
gen: 82
gen: 83
gen: 84
gen: 85
gen: 86
gen: 87
gen: 88
gen: 89
gen: 90
gen: 91
gen: 92
gen: 93
gen: 94
gen: 95
gen: 96
gen: 97
gen: 98
gen: 99
gen: 100
gen: 101
gen: 102
gen: 103
gen: 104
gen: 105
gen: 106
gen: 107
gen: 108
gen: 109
gen: 110
gen: 111
gen: 112
gen: 113


KeyboardInterrupt: 

In [ ]:
for i in data:
    x1, x2, x3, y = i[0], i[1], i[2], i[3]

    out4 = (x1 * final_rec['data']['w14']) + (x2 * final_rec['data']['w24']) + (x3 * final_rec['data']['w34']) + final_rec['data']['o4']
    out4 = sigmoid(out4)

    out5 = (x1 * final_rec['data']['w15']) + (x2 * final_rec['data']['w25']) + (x3 * final_rec['data']['w35']) + final_rec['data']['o5']
    out5 = sigmoid(out5)

    out6 = (out4 * final_rec['data']['w46']) + (out5 * final_rec['data']['w56']) + final_rec['data']['o6']
    out6 =  sigmoid(out6)

    final_ansr.append(out6)

In [ ]:
# final_ansr=final_ansr[-len(data):]
df['Y-ENN'] = pd.DataFrame(final_ansr)
df.to_csv('NN-DATA2.csv', index=False)

In [ ]:
from sklearn.metrics import r2_score
r2_score(np.array(data)[:,3],df['Y-ENN'])

In [497]:
final_ansr

[]

In [ ]:
np.array(data)[:,3]

In [ ]:
-11.580005159157485